# WRIT 340E: Advanced Writing for Engineers

---
# Illumin Article

### Basics of Online Recommender Systems - How do companies like Netflix make recommendations?

By: Chengyi (Jeff) Chen

Chengyi (Jeff) Chen is a junior studying Computer Science and Business Administration at the University of Southern California. He can be contacted via email: chen364@usc.edu, or cell: (626) 716-8409. Keywords: [‘Recommender Systems’, ‘Collaborative Filtering’, ‘Matrix Factorization’, ‘Singular Value Decomposition’, ‘Cosine Similarity’]

### Abstract

*Recommender systems filter information from consumers to suggest products with the maximum likelihood of being purchased. As consumer data becomes more available, better and more personalized recommendations can and have been built to increase the user experience. This article seeks to provide a simplified tutorial of recommendation systems and how they have grown to make more personalized recommendations for users.*

### Recommender Systems

Let's start off with a simple example of a recommender system to gain some intuition as to how recommendations work:

Starbuck's Barista: "*Morning Tom! Your usual Americano?*"

Tom: "*Yes please!*"

Starbuck's Barista: "*How about a sandwhich to pair with your drink?*"

Recommendations are everywhere. And where there are recommendations, there must be a system to make those recommendations. Why didn't the Barista recommend another Americano to Tom? Or maybe another drink like an espresso? Maybe the Barista has identified that drinks and sandwiches are complements and tend to "go together" as observed from past transactions. But we can't let the Barista do all the work right? How do companies like Netflix do it? Let us discover how these multinational conglomerates are using recommender systems to the "grab all the profits from the attention economy" [5].

### Popularity

To gain some clues as to how Netflix recommends movies, let's examine its landing page:

<img src='https://s.yimg.com/ny/api/res/1.2/rF4wdwOU_lQ9C3qtlLsGlQ--~A/YXBwaWQ9aGlnaGxhbmRlcjtzbT0xO3c9ODAw/http://media.zenfs.com/en-US/homerun/motleyfool.com/e619c2f21f7be1a551d7a49371af2591' style='border: 5px solid black; border-radius: 5px;'/>

We observe that the first movies that Netflix recommends are the most popular videos. This is probably the simplest recommender system we can build. However, the definition of "popularity" differs between different companies and across products. In the eyes of Netflix, the most "popular" videos might be the ones with the highest number of views, or videos with the highest average ratings. This method, however, has several disadvantages. Most importantly, there is a lack of personalization. What if you're only interested in Comedy, or Horror? Specific genres of movies would probably appeal to you better. Space on a landing page is a highly valuable piece of real estate that you want to fill with videos that the user is likely to click on. In the eyes of Netflix, "Personalization enables us to find an audience even for relatively niche videos that would not make sense for broadcast TV models because their audiences would be too small to support significant advertising revenue, or to occupy a broadcast or cable channel time slot" [1]. So how do we make more personalized recommendations?

### Collaborative Filtering

<img src='http://www.programmersought.com/images/597/b32b14c3a81f02687d3e02f1be7cc7c5.png' style='border: 5px solid black; border-radius: 5px;'/>

Let's find a way to make more personalized recommendations to our users. To do this, we'll use **Collaborative Filtering**, which asserts that users who are similar, like similar products (movies in the case of Netflix), and hence, **collaborative**. Consequently, our job can be categorized into 2 perspectives - **User-based** or **Item-based**. Imagine we have 2 very similar users, Tom and Harry. **User-based** recommendations would involve recommending Harry new movies that Tom had given positive reviews for. **Item-based** recommendations would involve recommending movies similar to the ones that Harry had reviewed positively.

#### Memory-based Techniques: User-based and Item-based

But how do we find "similar" users or movies? In order to understand how we'll quantify what "similarity" will be, let's take a look at what type of data we'll have when trying to build a recommendation system.

$$
\begin{aligned}
X_{n\times m} &= 
\underbrace{
\begin{bmatrix}
x_{11} & x_{12} & \ldots & x_{1j} & \ldots & x_{1m} \\ 
x_{21} & x_{22} &        &        &        &        \\ 
x_{31} &        & \ddots &        &        &        \\ 
\vdots &        &        & x_{ij} &        & \vdots \\ 
       &        &        &        & \ddots &        \\ 
x_{n1} &        & \ldots &        &        & x_{nm} \\ 
\end{bmatrix}
}_{n\,\text{rows (users)}\,\times\,m\,\text{columns (items)}}
\end{aligned}
$$

Suppose that we're trying to build a simple recommendation system for Amazon. Each row of the data matrix above, $X$, would represent each user, while each column would represent each product on Amazon that the user **may or may not have rated**. The entries within the matrix are then the ratings / reviews a user has given for a specific product.

To give a concrete example, let's say we had:

| Users / Movies | Titanic | The Fault in our Stars | Star Wars |
| :------------: | :-----: | :--------------------: | :-------: |
| Tom            | 2       | 2                      | 8         |
| Jeremy         | 3       | 1                      | 9         |
| Sarah          | 7       | 8                      | 4         |

$X = \begin{bmatrix} 2 & 2 & 8 \\ 3 & 1 & 9 \\ 7 & 8 & 4 \\ \end{bmatrix}$ is then our data matrix, with each row representing a different user and each column a different movie. Tom has given a rating of 2 for *Titanic*, 2 for *The Fault in our Stars*, and 8 for *Star Wars*. Jeremy has given a rating of 3 for *Titanic*, 1 for *The Fault in our Stars*, and 9 for *Star Wars*. Sarah has given a rating of 7 for *Titanic*, 8 for *The Fault in our Stars*, and 4 for *Star Wars*. Our job now is to then define a **similarity metric**, a way to score how similar each pair of users or products are. A very common **similarity metric** is the cosine similarity, which is given by:

$$
\begin{aligned}
cos\theta &= \frac{\mathbf{\vec{a}} \cdot \mathbf{\vec{b}}}{\vert\vert\mathbf{\vec{a}}\vert\vert\vert\vert\mathbf{\vec{b}}\vert\vert},\, 0 \leq cos(\theta) \leq 1
\end{aligned}
$$

For example, to implement User-based collaborative filtering, for every possible pair of users $a$ and $b$ (represented by the row vectors $\mathbf{\vec{a}} = \begin{bmatrix} x_{a1} & x_{a2} & x_{a3} & \ldots & x_{am} \end{bmatrix}$ and $\mathbf{\vec{b}} = \begin{bmatrix} x_{b1} & x_{b2} & x_{b3} & \ldots & x_{bm} \end{bmatrix}$), the cosine similarity is as follows:

$$
\begin{aligned}
cos(\theta) &= \frac{\begin{bmatrix} x_{a1} & x_{a2} & x_{a3} & \ldots & x_{am} \end{bmatrix} \cdot \begin{bmatrix} x_{b1} \\ x_{b2} \\ x_{b3} \\ \vdots \\ x_{bm} \end{bmatrix}}{\sqrt{x_{a1}^2 + x_{a2}^2 + x_{a3}^2 + \ldots + x_{am}^2}\sqrt{x_{b1}^2 + x_{b2}^2 + x_{b3}^2 + \ldots + x_{bm}^2}} \\
&= \frac{x_{a1}x_{b1} + x_{a2}x_{b2} + x_{a3}x_{b3} + \ldots + x_{am}x_{bm}}{\sqrt{x_{a1}^2 + x_{a2}^2 + x_{a3}^2 + \ldots + x_{am}^2}\sqrt{x_{b1}^2 + x_{b2}^2 + x_{b3}^2 + \ldots + x_{bm}^2}}
\end{aligned}
$$

Using the data matrix $X$ from above, let's calculate the cosine similarity between Tom and Jeremy, and between Tom and Sarah.

Tom and Jeremy:
$$
\begin{aligned}
cos(\theta) 
&= 
\frac{\begin{bmatrix} 
    2 & 2 & 8 \\ 
\end{bmatrix} 
\cdot 
\begin{bmatrix} 
    3 \\ 1 \\  9 \\ 
\end{bmatrix}}{\sqrt{2^2 + 2^2 + 8^2}\sqrt{3^2 + 1^2 + 9^2}} \\
&= \frac{(2\times 3) + (2\times 1) + (8\times 9)}{\sqrt{2^2 + 2^2 + 8^2}\sqrt{3^2 + 1^2 + 9^2}} \\
&= \frac{6 + 2 + 72}{\sqrt{4 + 4 + 64}\sqrt{9 + 1 + 81}} \\
&= \frac{80}{72 \times 91} \\
&= 0.0122 \\
\end{aligned}
$$

Tom and Sarah:
$$
\begin{aligned}
cos(\theta) 
&= 
\frac{\begin{bmatrix} 
    2 & 2 & 8 \\ 
\end{bmatrix} 
\cdot 
\begin{bmatrix} 
    7 \\ 8 \\  4 \\ 
\end{bmatrix}}{\sqrt{2^2 + 2^2 + 8^2}\sqrt{7^2 + 8^2 + 4^2}} \\
&= \frac{(2\times 7) + (2\times 8) + (8\times 4)}{\sqrt{2^2 + 2^2 + 8^2}\sqrt{7^2 + 8^2 + 4^2}} \\
&= \frac{14 + 16 + 32}{\sqrt{4 + 4 + 64}\sqrt{49 + 64 + 16}} \\
&= \frac{62}{72 \times 129} \\
&= 0.0067 \\
\end{aligned}
$$

We observe that Tom and Jeremy has a higher cosine similarity of 0.0122 to Tom and Sarah's cosine similarity of 0.0067. This means that Tom is more similar to Jeremy. By the assertion of Collaborative Filtering (similar users like similar products), we'll recommend Tom movies that he has never seen before, but Jeremy has given a positive review for.

In reality, we'll calculate the cosine similarity for each pair of users and store the similarities in **memory**, which is also the reason why we categorize user-based and item-based collaborative filtering as memory-based techniques. When we want to recommend a product to a user like Tom, we'll sort all the users that have the highest cosine similiarity to Tom in decreasing order and sequentially find movies that the most similar users, also known as Tom's **nearest neighbors**, have reviewed positively and recommend them to Tom [3].

However, **Scalability** can be a problem with these memory-based collaborative filtering techniques. Because the number of computations grows with both the number of users and products, we'll need more scalable solutions like model-based techniques such as matrix factorization [7]. 

#### Model-based Techniques: Matrix Factorization 

Introducing Singular Value Decomposition (SVD):

$$
\begin{aligned}
X_{n\times m} 
&= 
U_{n\times r} \cdot S_{r\times r} \cdot V^\top_{r\times m} \\
\begin{bmatrix}
x_{11} & x_{12} & \ldots & x_{1m} \\ 
x_{21} & x_{22} &        &        \\ 
\vdots &        & x_{ij} & \vdots \\
x_{n1} &        &        & x_{nm} \\ 
\end{bmatrix} 
&=
\begin{bmatrix}
u_{11} & u_{12} & \ldots & u_{1r} \\ 
u_{21} & u_{22} &        &        \\ 
\vdots &        & u_{ij} & \vdots \\
u_{n1} &        &        & u_{nr} \\ 
\end{bmatrix} 
\cdot
\begin{bmatrix}
s_{11} & 0      & \ldots & 0      \\ 
0      & s_{22} &        &        \\ 
0      &        & s_{ii} &        \\ 
\vdots &        &        & \vdots \\
0      &        &        & s_{rr} \\ 
\end{bmatrix} 
\cdot
\begin{bmatrix}
v_{11} & v_{12} & \ldots & v_{1m} \\ 
v_{21} & v_{22} &        &        \\ 
\vdots &        & v_{ij} & \vdots \\
v_{r1} &        &        & v_{rm} \\ 
\end{bmatrix} 
\end{aligned}
$$

SVD is a matrix factorization technique that splits a matrix $X$ into 3 others, $U$, $S$, and $V^\top$. Let's suppose again that we're trying to recommend movies to users on Netflix, and we were given the same data about Tom, Jeremy, and Sarah as before:

$$
\begin{aligned}
X_{n\times m} 
&= 
U_{n\times r} \cdot S_{r\times r} \cdot V^\top_{r\times m} \\
\begin{bmatrix}
    2 & 2 & 8 \\
    3 & 1 & 9 \\
    7 & 8 & 4 \\
\end{bmatrix}
&=
\begin{bmatrix}
    -0.5 & -0.4 & -0.8 \\
    -0.6 & -0.5 & 0.6 \\
    -0.6 & 0.8 & 0.0 \\
\end{bmatrix}
\cdot
\begin{bmatrix}
    15.4 & 0 & 0 \\
    0 & 7.4 & 0 \\
    0 & 0 & 0.9 \\
\end{bmatrix}
\cdot
\begin{bmatrix}
    -0.5 & -0.4 & -0.8 \\
    0.4 & 0.7 & -0.6 \\
    0.8 & -0.6 & -0.1 \\
\end{bmatrix}
\end{aligned}
$$

$U = \begin{bmatrix} -0.5 & -0.4 & -0.8 \\ -0.6 & -0.5 & 0.6 \\ -0.6 & 0.8 & 0.0 \\ \end{bmatrix}$ is the user matrix, whose rows represents how much users "feel" for specific latent factors, the columns of the matrix. But what are latent factors? They are the unobservable attributes of movies / items / products that we are recommending. In this case, we can interpret latent factors as the genres of the movies that we're recommending - Comedy, Romance, Action. Jeremy feels the most strongly for Action as it has a score of 0.6, while he does not care about the amount of Comedy or Romance in a movie. This also explains why he gave *Star Wars* such a high score of 9 as compared to *Titanic* and *The Fault in our Stars*. We might possibly recommend him Action-heavy movies in the future.

$V^\top = \begin{bmatrix} -0.5 & -0.4 & -0.8 \\ 0.4 & 0.7 & -0.6 \\ 0.8 & -0.6 & -0.1 \\ \end{bmatrix}$ is the movie matrix, whose columns represents the amount of each genre - Comedy, Romance, Action - is in each movie. *The Fault in our Stars*, the middle column has a very high "amount" of Romance (as observed by the 0.7 score), a little bit of Comedy (as observed by the -0.4 score), and very little Action (as observed by the -0.6 score).

$S = \begin{bmatrix} 15.4 & 0 & 0 \\ 0 & 7.4 & 0 \\ 0 & 0 & 0.9 \\ \end{bmatrix}$. The diagonal entries in $S$ actually represent the weights of latent factors. 15.4 would correspond to the "weight" of the amount of Comedy in the movie, 7.4 would correspond to the "weight" of Romance in the movie, and 0.9 would correspond to the "weight" of Action in the movie. 

There are 2 main advantages when using latent factor models like SVD. Firstly, they are more scalable than memory-based techniques [7]. Secondly, they offer highly expressive ability to describe various aspects of the data and thus, tend to provide more accurate results than neighborhood models [2].

However, there are still some issues that come with this approach. Firstly, the results might not make any sense. In our $U$ matrix above, we can see that Tom's score for Action is -0.8 when he rated Star Wars, an action-packed movie, an 8! The problem is that we needed to account for the weights of the latent factors (genres) in the $S$ matrix as well, meaning that our analysis above is not exactly correct. Furthermore, remember how we said at the start that users **may or may not** have rated a particular item? In platforms such as Amazon and Netflix, there are an overwhelmingly larger number of products / movies that users have not rated as opposed to the number of products they have purchased / watched. This is also known as **Sparsity**. This problem exists in both memory-based techniques and simpler model-based techniques like SVD. Because of how unlikely it is to find users that have both purchased very similar items due to the sheer mass of products available for purchase in Amazon, the idea of using "similarity" between users or items degrades the usefulness of memory-based techniques and making recommendations extremely difficult, and the accuracy of the output to be really low [7]. **Sparsity** renders SVD unusable because SVD requires the matrix to be complete. However, not all is lost with SVD as we can still modify it to make it more interpretable and more resilient to **Sparsity**.

### FunkSVD

Unlike SVD, FunkSVD (Created by Simon Funk as part of the Netflix Prize Competition in 2007 [8]) splits the data matrix into 2 matrices:

$$
\begin{aligned}
X_{n\times m} 
&= 
U_{n\times r} \cdot V^\top_{r\times m} \\
\end{aligned}
$$

Instead of finding 3 exact matrices that make up the original data matrix $X$ in SVD, FunkSVD uses optimization techniques to find the 2 approximate matrices that can best reconstruct the original data matrix $X$. The interpretation of the 2 matrices, $U$ and $V^\top$ remain the same as for SVD. This way however, we don't require the matrix $X$ to be at all complete, because we're approximating $U$ and $V^\top$, solving the problem of **Sparsity**. The 2 matrices are much more interpretable as well because we don't need to account for the middle $S$ matrix anymore.

### Future of Recommender Systems

We've only scratched the surface on the types of Recommender systems out there. The levels of complexity that can be incorporated into recommender systems only increase from here. For example, recommender systems that have exploited "temporal context has been proved to be an effective approach to improve recommendation performance, as shown, e.g. in the Netflix Prize competition" [4]. Also, deep learning based recommender systems have been gaining significant attention because of their abilities to capture "intricate relationships within the data itself, from abundant accessible data sources such as contextual, textual and visual information" [6]. However, there are also other less obvious factors that we need to account for in building recommender systems, such as the **serendipity** factor when it comes to recommendations. A recommendation is not just about how accurate it is; recommendations that are unexpectedly good are also important in providing users with that "wow" experience. As the complexity of these models grow over time and "absorb" even more data, these recommender systems will certainly become more powerful in predicting our needs and wants much further into the future than we could possibly imagine, enhancing our user experience of these products ever more delightfully.

### References

[1] Gomez-Uribe, Carlos; Hunt, Neil. (2015). *The Netflix Recommender System: Algorithms, Business Value, and Innovation* [Online]. Available: http://delivery.acm.org/10.1145/2850000/2843948/a13-gomez-uribe.pdf?ip=68.180.70.23&id=2843948&acc=OA&key=4D4702B0C3E38B35%2E4D4702B0C3E38B35%2E4D4702B0C3E38B35%2EE5B8A747884E71D5&__acm__=1567825268_41bb18002e4f22845f3426c1f2b54cd9

[2] Yehuda Koren. (2010). *Factor in the neighbors: Scalable and accurate collaborative filtering* [Online]. Available: http://courses.ischool.berkeley.edu/i290-dm/s11/SECURE/a1-koren.pdf

[3] Leidy Esperanza Molina Fernandez. (2018). *Recommendation System for Netflix* [Online]. Available: https://beta.vu.nl/nl/Images/werkstuk-fernandez_tcm235-874624.pdf

[4] Pedro G. Campo, Fernando Díez, Iván Cantador. (2013). *Time-aware recommender systems: a comprehensive survey and analysis of existing evaluation protocols* [Online]. Available: https://link.springer.com/article/10.1007/s11257-012-9136-x

[5] Matthew Hindman. (2018). *The Internet Trap: How the Digital Economy Builds Monopolies and Undermines Democracy* [Online]. Available: https://www.jstor.org/stable/j.ctv36zrf8

[6] Shuai Zhang, Lina Yao, Aixin Sun, Yi Tay. (2018). *Deep Learning based Recommender System: A Survey and New Perspectives* [Online]. Available: https://arxiv.org/pdf/1707.07435.pdf

[7] Sarwar, Badrul et al. (2001). *Item-based Collaborative Filtering Recommendation Algorithms* [Online]. Available: https://dl.acm.org/citation.cfm?doid=371920.372071

[8] Gregory Piatetsky. (2007). *Interview with Simon Funk* [Online]. Available: https://www.kdd.org/exploration_files/simon-funk-explorations.pdf